In [1]:
%load_ext autoreload
%autoreload 2

from statsmodels.base.model import GenericLikelihoodModel

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys
import re

from scipy.interpolate import make_interp_spline, BSpline
from scipy.stats import multivariate_normal

sys.path.append("../")
import vuong_tests11 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=9):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:
def gen_data(nobs=1000, a=0.25, num_params=4): #note this was originally 9?
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=9)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
import numpy as np
import vuong_tests11 as vuong_tests_fast   # <-- use your actual module name

# uses your existing setup_shi(yn, xn, num_params=9)
# and your existing gen_data(nobs=..., a=..., num_params=...)

def run_one_beta(
    beta=0.0,
    nobs=100,
    epsilon=0.5,
    nsims=700,
    trials=300,                 # bootstrap reps (set lower for speed)
    skip_boot=False,
    skip_shi=True,              # Shi can be expensive; turn on if you need it
    data_tuned_epsilon=True,
    adapt_c=True,
    alpha=0.05,
    biascorrect=True
):
    # fixed DGP at this beta
    gen_data_one = lambda: gen_data(nobs=nobs, a=beta, num_params=9)

    # run Monte Carlo once (for this beta only)
    mc_out = vuong_tests_fast.monte_carlo(
        total=nsims,
        gen_data=gen_data_one,
        setup_shi=setup_shi,
        trials=trials,
        skip_boot=skip_boot,
        skip_shi=skip_shi,
        epsilon=epsilon,
        data_tuned_epsilon=data_tuned_epsilon,
        adapt_c=adapt_c,
        alpha=alpha,
        print_stuff=False,
        biascorrect=True
    )

    # pretty table (your helper)
    vuong_tests_fast.print_mc(mc_out, data_tuned_epsilon=data_tuned_epsilon)

    # if you specifically want the pairwise bootstrap shares (like your print(beta, boot2)):
    reg, twostep, sw, boot1, boot2, sw_opt, boot3, shi, llr, std, omega = mc_out
    if not skip_boot:
        print("\nPairwise bootstrap (boot2) = [no-select, model1, model2]:")
        print(boot2)

    return mc_out




In [5]:
# Example: run near 0 where size distortion shows up
epsilon = 0.5
nobs    = 100

mc0 = run_one_beta(
    beta=0.0,          # try 0.0, 0.02, 0.05, 0.1, etc.
    nobs=nobs,
    epsilon=epsilon,
    nsims=25,
    trials=300,        # increase if you want more stable bootstrap critical values
    skip_boot=False,
    skip_shi=True,
    data_tuned_epsilon=False,
    biascorrect=True
)


llr -4.250527468775396 -10.475431954287401 0.5
omega 0.10238487909393722 1.118859301851594 0.2790135812756071 0.31997637271201324 0.5729846673618815
llr_reg,omega_reg -9.488243445919096 0.5729846673618815 4.13422813066052
mean=-0.542, cv_lower=-2.378, stat_obs=-0.934, cv_upper=1.504
returned out = 0
llr -6.731137009354749 -4.98224513222992 0.5
omega 0.14724280146511654 0.7676521077420007 0.32594416564914447 0.3837222973259653 0.5979663338111669
llr_reg,omega_reg -9.22225957546971 0.5979663338111669 4.153453839839429
mean=-0.411, cv_lower=-2.377, stat_obs=-0.848, cv_upper=1.820
returned out = 0
llr -3.1269608827282083 6.378460905214709 0.5
omega 0.10618261195864039 0.28073869449644584 0.3189744645011534 0.32585673532802784 0.4839814694930587
llr_reg,omega_reg 0.062269569879146225 0.4839814694930587 3.9164667807399987
mean=-0.910, cv_lower=-3.368, stat_obs=0.822, cv_upper=1.306
returned out = 0
llr -6.638464033364813 2.6125736340777763 0.5
omega 0.20646525411268457 0.48221924241856656 0.